# Monitoring AI Models For Bias and Fairness with segmentation

In [1]:
!ls

sample_data


In [24]:
# Installing whylogs (WhyLabs.ai is an AI observability platform that prevents data & model performance degradation by allowing you to monitor your data and machine learning models in production)
!pip install whylogs[viz]==1.3.0

#⚖️ ML Monitoring for Bias: Monitoring for Bias & Fairness with Tracing



In [4]:
# Imports
import whylogs as why
import numpy as np
import pandas as pd
import datetime
import os

from sklearn.model_selection import train_test_split


from sklearn.datasets import load_iris

# Set to show all columns in dataframe
pd.set_option("display.max_columns", None)

In [5]:
# Load iris data as dataframe(df)
data_iris = load_iris(as_frame=True)

# List names in dataset
print(list(data_iris.target_names))
print(list(data_iris.data))

['setosa', 'versicolor', 'virginica']
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


## Train a Machine Learning Model (quickly)

In [6]:
# Train baseline Model
# KNN Model
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier(n_neighbors=5)

# Create featurex and target data varaible
X, y = data_iris.data, data_iris.target

#create train & test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=42,
                                                    stratify=y)
# Train model
knn.fit(X_train, y_train)

# Predict the labels on test data sset
y_pred = knn.predict(X_test)

# Print model accuracy
knn.score(X_test, y_test)


0.9777777777777777

### KNN intuition

Just a little bit of intuition how kNN models work.

This will be helpful for troubleshooting some issues later!

Iris data plotted by:

`x = 'sepal length (cm)', y = 'petal width (cm)'`

![](https://github.com/sagecodes/intro-machine-learning/raw/master/irisknn.png)

## Import data batches

In [7]:
 # Import data batches
url = 'https://raw.githubusercontent.com/sagecodes/sythetic_iris_data/main/iris_15_statefl_1.csv'
data_batch_1 = pd.read_csv(url)

url2 = 'https://raw.githubusercontent.com/sagecodes/sythetic_iris_data/main/iris_16_statefl_1.csv'
data_batch_2 = pd.read_csv(url2)

url3 = 'https://raw.githubusercontent.com/sagecodes/sythetic_iris_data/main/iris_17_statefl_1.csv'
data_batch_3 = pd.read_csv(url3)

url4 = 'https://raw.githubusercontent.com/sagecodes/sythetic_iris_data/main/iris_18_statefl_1.csv'
data_batch_4 = pd.read_csv(url4)

url5 = 'https://raw.githubusercontent.com/sagecodes/sythetic_iris_data/main/iris_19_statefl_1.csv'
data_batch_5 = pd.read_csv(url5)

url6 = 'https://raw.githubusercontent.com/sagecodes/sythetic_iris_data/main/iris_20_statefl_1.csv'
data_batch_6 = pd.read_csv(url6)

url7 = 'https://raw.githubusercontent.com/sagecodes/sythetic_iris_data/main/iris_21_statefl_1.csv'
data_batch_7 = pd.read_csv(url7)

# iris feature names
feature_names = ['sepal length (cm)', 'sepal width (cm)','petal length (cm)','petal width (cm)', 'state']

# separate targets
X_batch_1 = data_batch_1[feature_names]
X_batch_2 = data_batch_2[feature_names]
X_batch_3 = data_batch_3[feature_names]
X_batch_4 = data_batch_4[feature_names]
X_batch_5 = data_batch_5[feature_names]
X_batch_6 = data_batch_6[feature_names]
X_batch_7 = data_batch_7[feature_names]

# We'll save the target values for later!
y_batch_1 = data_batch_1['target']
y_batch_2 = data_batch_2['target']
y_batch_3 = data_batch_3['target']
y_batch_4 = data_batch_4['target']
y_batch_5 = data_batch_5['target']
y_batch_6 = data_batch_6['target']
y_batch_7 = data_batch_7['target']


# create list of our batches
dfs = [X_batch_1, X_batch_4, X_batch_5, X_batch_6, X_batch_2, X_batch_3, X_batch_7]

df_target = [y_batch_1, y_batch_4, y_batch_5, y_batch_6, y_batch_2, y_batch_3, y_batch_7]


In [8]:
dfs[0].head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),state
0,4.9,4.0,1.6,0.3,Washington
1,4.9,3.5,1.3,0.1,Washington
2,5.9,3.0,5.1,1.3,Washington
3,5.2,3.3,1.6,0.3,Washington
4,4.6,3.2,1.2,0.3,Washington



## Creating profiles with whylogs


Profiles generated with whylogs are:

- Secure
- Efficient
- Customizable
- Mergeable

In [25]:
# create profile
profile1 = why.log(X_batch_1)

profile_view1 = profile1.view()
profile_view1.to_pandas()

,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/inf,counts/n,counts/nan,counts/null,distribution/max,distribution/mean,distribution/median,distribution/min,distribution/n,distribution/q_01,distribution/q_05,distribution/q_10,distribution/q_25,distribution/q_75,distribution/q_90,distribution/q_95,distribution/q_99,distribution/stddev,frequent_items/frequent_strings,type,types/boolean,types/fractional,types/integral,types/object,types/string,types/tensor
column,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
cls_output,3.000000,3.0,3.000150,0,150,0,0,NaN,0.000000,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,"[FrequentItem(value='virginica', est=51, upper...",SummaryType.COLUMN,0,0,0,0,150,0
ground_truth,3.000000,3.0,3.000150,0,150,0,0,NaN,0.000000,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,"[FrequentItem(value='virginica', est=50, upper...",SummaryType.COLUMN,0,0,0,0,150,0
petal length (cm),49.000006,49.0,49.002452,0,150,0,0,7.5,4.041333,4.3,1.1,150,1.1,1.3,1.5,2.4,5.4,6.2,6.8,7.5,1.776664,NaN,SummaryType.COLUMN,0,150,0,0,0,0
petal width (cm),24.000001,24.0,24.001200,0,150,0,0,2.7,1.207333,1.3,0.0,150,0.1,0.1,0.2,0.3,1.9,2.2,2.4,2.5,0.769651,NaN,SummaryType.COLUMN,0,150,0,0,0,0
prob_output,3.000000,3.0,3.000150,0,150,0,0,1.0,0.973333,1.0,0.6,150,0.6,0.8,1.0,1.0,1.0,1.0,1.0,1.0,0.088740,NaN,SummaryType.COLUMN,0,150,0,0,0,0
sepal length (cm),33.000003,33.0,33.001650,0,150,0,0,7.7,5.856000,5.8,4.2,150,4.2,4.7,4.9,5.1,6.5,7.1,7.2,7.7,0.847775,NaN,SummaryType.COLUMN,0,150,0,0,0,0
sepal width (cm),19.000001,19.0,19.000950,0,150,0,0,4.0,2.996000,2.9,2.2,150,2.2,2.3,2.4,2.7,3.3,3.6,3.8,4.0,0.448395,NaN,SummaryType.COLUMN,0,150,0,0,0,0
state,3.000000,3.0,3.000150,0,150,0,0,NaN,0.000000,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,"[FrequentItem(value='Washington', est=50, uppe...",SummaryType.COLUMN,0,0,0,0,150,0


## Writing data profiles to to WhyLabs

![](https://camo.githubusercontent.com/8e9cc18b64b157d4569fa6ed2bd5152200ee7bb1a11e54f858f923a4be635f90/68747470733a2f2f7768796c6162732e61692f5f6e6578742f696d6167653f75726c3d6874747073253341253246253246636f6e74656e742e7768796c6162732e6169253246636f6e74656e74253246696d616765732532463230323225324631312532464672616d652d363839392d2d312d2e706e6726773d3331323026713d3735)


In [26]:
# set authentication & project keys
os.environ["WHYLABS_DEFAULT_ORG_ID"] = 'org-ZSqhdy'
os.environ["WHYLABS_API_KEY"] = 'IxGGWOPQDD.g90N0Dg1p3Wona7rwxqPrZzTRH6uNeWjtfDWlcxyVM5ee3AfT2Lgd:org-ZSqhdy'
os.environ["WHYLABS_DEFAULT_DATASET_ID"] = 'model-2'

### write a single profile
```
profile = why.log(df)
profile.writer("whylabs").write()
```

### Create dataframe with model predictions

In [27]:
# Get predictions with model & append to df
pred_dfs = dfs

class_names = ['setosa', 'versicolor', 'virginica']

for i, df in enumerate(pred_dfs):
    y_pred = knn.predict(df.iloc[:, :4])
    y_prob = knn.predict_proba(df.iloc[:, :4])
    pred_scores = []
    pred_classes = []

    for pred in y_pred:
      pred_classes.append(class_names[pred])
    df['cls_output'] = pred_classes
    for prob in y_prob:
      pred_scores.append(max(prob))
    df['prob_output'] = pred_scores

In [28]:
pred_dfs[-1]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),state,cls_output,prob_output,ground_truth
0,5.5,2.9,4.4,1.2,Washington,versicolor,1.0,versicolor
1,4.7,3.1,1.3,0.3,Washington,setosa,1.0,setosa
2,4.7,3.0,1.4,0.2,Washington,setosa,1.0,setosa
3,5.2,3.7,1.4,0.3,Washington,setosa,1.0,setosa
4,6.8,3.2,6.2,1.6,Washington,virginica,1.0,virginica
...,...,...,...,...,...,...,...,...
145,5.9,3.0,5.8,2.6,Missouri,virginica,1.0,virginica
146,4.8,2.3,3.7,1.4,Missouri,versicolor,1.0,versicolor
147,5.3,2.7,4.3,1.3,Missouri,versicolor,1.0,versicolor
148,5.8,3.2,5.7,1.9,Missouri,virginica,1.0,virginica


### Backfilling data in WhyLabs

In [29]:
from whylogs.core.schema import DatasetSchema
from whylogs.core.segmentation_partition import segment_on_column

In [30]:
# back fill 1 day per batch
for i, df in enumerate(pred_dfs):
    # walking backwards. Each dataset has to map to a date to show up as a different batch in WhyLabs
    dt = datetime.datetime.now(tz=datetime.timezone.utc) - datetime.timedelta(days=i)

    # create profile for each batch of data
    profile = why.log(df, schema=DatasetSchema(segments=segment_on_column("state")))

    # set the dataset timestamp for the profile
    profile.set_dataset_timestamp(dt)
    # write the profile to the WhyLabs platform
    profile.writer("whylabs").write()

###Classification Performance Metrics

In [31]:
# Append ground truth data to dataframe
for i, df in enumerate(pred_dfs):
    df['ground_truth'] = df_target[i]

In [32]:
pred_dfs[0]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),state,cls_output,prob_output,ground_truth
0,4.9,4.0,1.6,0.3,Washington,setosa,1.0,setosa
1,4.9,3.5,1.3,0.1,Washington,setosa,1.0,setosa
2,5.9,3.0,5.1,1.3,Washington,versicolor,0.8,versicolor
3,5.2,3.3,1.6,0.3,Washington,setosa,1.0,setosa
4,4.6,3.2,1.2,0.3,Washington,setosa,1.0,setosa
...,...,...,...,...,...,...,...,...
145,6.0,2.9,6.6,2.0,Missouri,virginica,1.0,virginica
146,5.3,3.0,1.6,0.1,Missouri,setosa,1.0,setosa
147,5.6,2.3,4.2,1.2,Missouri,versicolor,1.0,versicolor
148,5.3,3.1,1.4,0.3,Missouri,setosa,1.0,setosa


In [33]:
from whylogs import log_classification_metrics
# from whylogs.core.schema import DatasetSchema
# from whylogs.core.segmentation_partition import segment_on_column

In [34]:
for i, df in enumerate(pred_dfs):

  segmented_classification_results = log_classification_metrics(
    df,
    target_column = "ground_truth",
    prediction_column = "cls_output",
    schema = DatasetSchema(segments=segment_on_column("state"))
  )
   # walking backwards. Each dataset has to map to a date to show up as a different batch in WhyLabs
  dt = datetime.datetime.now(tz=datetime.timezone.utc) - datetime.timedelta(days=i)

  # profile = segmented_classification_results.profile()
  segmented_classification_results.set_dataset_timestamp(dt)

  segmented_classification_results.writer("whylabs").write()